In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [15]:
train_folders = ['D:/Project/Thermal images testing/flir_17_Sept_2013/train/positives',
                 'D:/Project/Thermal images testing/flir_17_Sept_2013/train/negatives']

In [16]:
test_folders = ['D:/Project/Thermal images testing/flir_17_Sept_2013/test_created_by_me/positives', 
                'D:/Project/Thermal images testing/flir_17_Sept_2013/test_created_by_me/negatives']

In [18]:
pos_train_image_width = 64
pos_train_image_height = 128

neg_train_image_width = 324
neg_train_image_height = 256

test_image_width = 324
test_image_height = 256

pixel_depth = 8.0  # Number of levels per pixel.

def load_class(folder, min_num_images, im_height, im_width, im_height_alt , im_width_alt):
  """Load the data for a single letter label."""

  image_files = os.listdir(folder)
  test_image = (ndimage.imread(os.path.join(folder, image_files[0])).astype(float) - 
                    pixel_depth / 2) / pixel_depth
  if test_image.shape != (im_height, im_width):
        im_height = im_height_alt
        im_width = im_width_alt
    
  dataset = np.ndarray(shape=(len(image_files), im_height, im_width),
                         dtype=np.float32)
  
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (im_height, im_width):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, im_height, im_width, im_height_alt, im_width_alt, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_class(folder, min_num_images_per_class, im_height, im_width, im_height_alt, im_width_alt)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 90, pos_train_image_height, pos_train_image_width, 
                              neg_train_image_height, neg_train_image_width)
test_datasets = maybe_pickle(test_folders, 90, test_image_height, test_image_width, test_image_height, test_image_width)


D:/Project/Thermal images testing/flir_17_Sept_2013/train/positives.pickle already present - Skipping pickling.
Pickling D:/Project/Thermal images testing/flir_17_Sept_2013/train/negatives.pickle.
D:/Project/Thermal images testing/flir_17_Sept_2013/train/negatives
Full dataset tensor: (92L, 256L, 324L)
Mean: 13.068
Standard deviation: 6.17157
Pickling D:/Project/Thermal images testing/flir_17_Sept_2013/test_created_by_me/positives.pickle.
D:/Project/Thermal images testing/flir_17_Sept_2013/test_created_by_me/positives
Full dataset tensor: (778L, 256L, 324L)
Mean: 9.93009
Standard deviation: 3.95065
Pickling D:/Project/Thermal images testing/flir_17_Sept_2013/test_created_by_me/negatives.pickle.
D:/Project/Thermal images testing/flir_17_Sept_2013/test_created_by_me/negatives
Full dataset tensor: (169L, 256L, 324L)
Mean: 10.8741
Standard deviation: 2.83364
